<a href="https://colab.research.google.com/github/rtajeong/AI_Cluster/blob/main/lab50_bag_of_word.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bag of Word (BOW)
- Document-Term Matrix and Tfidf Matrix

뉴스기사 분석
- 문서-단어 행렬의 예시 (Document-term Matrix)
- 빅카인즈 뉴스기사 데이터 활용(https://www.bigkinds.or.kr/): 한국언론진흥재단이 운영하는 사이트로
  국내 언론사들 뉴스기사의 분야별 키워드, 통계, 이슈 등을 제공한다.
- 여기서는 2019년 뉴스 데이터 사용

In [1]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [2]:
# 뉴스 데이터 가져오기
!curl -L https://bit.ly/2X7UON2 -o news.xlsx

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   162  100   162    0     0   1883      0 --:--:-- --:--:-- --:--:--  1883
100   158    0   158    0     0    524      0 --:--:-- --:--:-- --:--:--   524
100   318  100   318    0     0    591      0 --:--:-- --:--:-- --:--:--     0
100  1076    0  1076    0     0   1545      0 --:--:-- --:--:-- --:--:--  1545
100 1789k  100 1789k    0     0  1735k      0  0:00:01  0:00:01 --:--:-- 1735k


In [3]:
# !dir

In [4]:
news_all = pd.read_excel("news.xlsx")
news_all.columns

Index(['뉴스 식별자', '일자', '언론사', '기고자', '제목', '통합 분류1', '통합 분류2', '통합 분류3',
       '사건/사고 분류1', '사건/사고 분류2', '사건/사고 분류3', '인물', '위치', '기관', '키워드', '특성추출',
       '본문', 'URL', '분석제외 여부'],
      dtype='object')

In [5]:
news_all.head(2).T

,0,1
뉴스 식별자,1.5004e+06,1.2002e+06
일자,20190228,20190228
언론사,국제신문,경인일보
기고자,김태경 기자 tgkim@kookje.co.kr,조영상
제목,빅딜이냐 스몰딜이냐 ‘영변 핵+α’ 로드맵 도출이 관건,"하노이 도착후 실무대표단 청취 김정은, 북미회담 성공 '잰걸음'"
통합 분류1,정치>북한,국제>국제일반
통합 분류2,정치>외교,정치>북한
통합 분류3,NaN,NaN
사건/사고 분류1,NaN,NaN
사건/사고 분류2,NaN,NaN


In [6]:
news_text = news_all['본문']
news_text[:5]

0    - 비핵화 수준 상응 조치 놓고\n- 양국 협상팀 막판까지 ‘밀당’\n- 1차 때와...
1    김정은 국무위원장이 27일 시작되는 제2차 북미정상회담 성공을 위해 심혈을 기울이고...
2    북미가 처음으로 정상 간 단독회담과 만찬을 가지며 또다시 새로운 역사 창조에 나섰다...
3    지난해 9월 남북정상회담 당시 리선권 북한 조국평화통일위원장의 '냉면' 발언으로 정...
4    지자체 민간 교류 활성화 대부분 \n여, 부처간 논의 예산 지원 확대 \n야, 사업...
Name: 본문, dtype: object

In [7]:
len(news_text[0])

200

In [8]:
type(news_text)

pandas.core.series.Series

In [9]:
cv = CountVectorizer()
dtm = cv.fit_transform(news_text)
df = pd.DataFrame(pd.DataFrame(dtm.toarray(), columns = cv.get_feature_names()))
df[:5]

,00,000원을,001420,001550,002100,005690,01,017800,02,025860,025980,03,04,050090,054300,06,07,070960,08,09,0px,0시,0포인트,10,1000여개에,100년,100년을,100년전,100배,100일,100일을,100주년,100주년에,100주년을,100주년의,100주년이,10cm,10년,10년의,10대,...,흐름을,흐름이다,흔드는,흔들고,흔들리는,흔들리지,흔들림,흔들림없는,흘러,흘러나오고,흘러나오는,흘렀다,흘린,흙수저,흡연하는,흥미로운,흥미로울,흥행,흥행에,흥행을,희망,희망도,희망으로,희망하고,희망하며,희망한다,희망합니다,희박할,희비나,희석됐고,힌국당,힘겨루기로,힘겨웠던,힘들다고,힘들어,힘들어지는,힘을,힘이,힘입어,靈山
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [11]:
df.shape    # 총 단어의 수가 16,1999 개

(1543, 16199)

In [12]:
# 어떤 단어들이 등장했는지 확인 - dict
len(cv.vocabulary_.items())

16199

In [13]:
# 문서-단어 행렬에서 가장 많이 등장하는 단어 찾기 (이것이 훨씬 간단)
df.sum()    # column-wise sum
pd.Series(df.sum()).idxmax()

'2차'

In [14]:
# DSAC 교재에 나타난 코드 (결과는 위와 동일)
top_word = []
for i in df.T.values:    # df.T.values 와 동일 (2-d array)
    top_word.append(sum(i.tolist()))
    
df.transpose().index[top_word.index(max(top_word))]

'2차'

In [15]:
# maximum 2000 개만 사용하도록 하기 위해서는?
cv = CountVectorizer(max_features=200)
dtm = cv.fit_transform(news_text)
df = pd.DataFrame(pd.DataFrame(dtm.toarray(), columns = cv.get_feature_names()))
print(df.shape)
df[:6]

(1543, 200)


,10일,11일,17일,18일,19일,1차,21일,24일,25일,26일,27,27일,28일,2월,2차,6월,6일,7일,8일,9일,the300,가능성이,가운데,개최,개최를,것으로,것을,것이라고,것이라는,경제,공식,관련,관련해,국무부,국무위원장과,국무위원장의,국무위원장이,국회,국회에서,기자,...,정상이,정상회담,정상회담에,정상회담에서,정상회담을,정상회담의,정상회담이,제2차,제재,주요,중국,중국을,지난,지난해,진행,청와대,청와대는,통해,트럼프,특파원,평화,폼페이오,하고,하노이,하노이에,하노이에서,한국당,한미,한반도,한반도의,함께,했다,헤럴드경제,현지시간,협상,협상을,홍영표,회담,회담을,회담이
0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,0,0,0,...,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [16]:
df.max(axis=1)

0       2
1       2
2       1
3       1
4       1
       ..
1538    2
1539    2
1540    1
1541    2
1542    2
Length: 1543, dtype: int64

In [30]:
tv = TfidfVectorizer(max_features=200)
dtm = tv.fit_transform(news_text.values)
df = pd.DataFrame(pd.DataFrame(dtm.toarray(), columns = tv.get_feature_names()))
print(df.shape)
df[:6]

(1543, 200)


,10일,11일,17일,18일,19일,1차,21일,24일,25일,26일,27,27일,28일,2월,2차,6월,6일,7일,8일,9일,the300,가능성이,가운데,개최,개최를,것으로,것을,것이라고,것이라는,경제,공식,관련,관련해,국무부,국무위원장과,국무위원장의,국무위원장이,국회,국회에서,기자,...,정상이,정상회담,정상회담에,정상회담에서,정상회담을,정상회담의,정상회담이,제2차,제재,주요,중국,중국을,지난,지난해,진행,청와대,청와대는,통해,트럼프,특파원,평화,폼페이오,하고,하노이,하노이에,하노이에서,한국당,한미,한반도,한반도의,함께,했다,헤럴드경제,현지시간,협상,협상을,홍영표,회담,회담을,회담이
0,0.0,0.0,0.0,0.0,0.0,0.346577,0.0,0.0,0.0,0.000000,0.0,0.304950,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.349417,0.0,0.00000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.387461,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.179136,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.287727,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.258984,0.0,0.242432,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.182286,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.43136,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.389187,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.285075,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.348002,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.398748,0.0,0.00000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.204426,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.274912,0.0,0.0,0.0,0.000000,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.00000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.521872,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.186464,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.442107,0.0,0.0,0.000000,0.0,0.00000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.39742,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.094297,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.157885,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.00000,0.0,0.0,0.126213,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0


# 연습

In [18]:
df2 = pd.DataFrame(np.arange(12).reshape(3,4)); df2
df2.values

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [19]:
df2.sum()

0    12
1    15
2    18
3    21
dtype: int64

In [20]:
df2.max()

0     8
1     9
2    10
3    11
dtype: int64

In [21]:
df2.idxmax(axis=1)

0    3
1    3
2    3
dtype: int64

In [22]:
df2.T.idxmax()

0    3
1    3
2    3
dtype: int64

In [23]:
for i in df2.values:
    print(sum(i))

6
22
38


In [24]:
tmp=[]
for i in df2.T.values:
    tmp.append(sum(i))

In [25]:
tmp

[12, 15, 18, 21]

In [26]:
tmp=np.array(df2.sum()); tmp

array([12, 15, 18, 21])

In [27]:
tmp_s = pd.Series(tmp)
tmp_s.idxmax()

3